# HVM Creep

**Hybrid Vitrimer Model — Retardation modes and vitrimer plastic creep**

## Physical Motivation

Creep experiments under constant applied stress reveal the **long-time flow**
behavior of vitrimers. Unlike conventional thermosets (which reach a fixed
deformation $\gamma = \sigma_0/G$), vitrimers exhibit **plastic creep** at
intermediate times as BER continuously redefines the stress-free reference
state. The creep compliance $J(t) = \gamma(t)/\sigma_0$ shows:

1. An immediate elastic jump from all three networks
2. Retardation as E and D networks progressively relax stress
3. A long-time plateau $J(\infty) \to 1/G_P$ set by the permanent network

This protocol is critical for predicting **reprocessing kinetics** and
**shape-memory programming** — applications where vitrimer materials undergo
slow deformation under sustained load.

> **Handbook:** See [HVM Protocol Derivations](../../docs/source/models/hvm/hvm_protocols.rst) for the creep ODE derivation. See [HVM Knowledge Extraction](../../docs/source/models/hvm/hvm_knowledge.rst) for creep in the fitting strategy (step 5).

## Learning Objectives

- Understand retardation modes in creep compliance $J(t)$
- Observe elastic jump at $t=0$ and long-time plateau $1/G_P$
- Analyze vitrimer plastic creep contribution
- Compare stress-dependent creep behavior (TST acceleration)

## Prerequisites

- **Notebook 01** (SAOS) — HVM architecture and parameter meanings

## Estimated Runtime

- ~2 minutes

In [ ]:
# Google Colab setup (run only if on Colab)
try:
    import google.colab
    IN_COLAB = True
    !pip install -q rheojax
except ImportError:
    IN_COLAB = False
    print("Running locally")

In [ ]:
# Imports
from rheojax.core.jax_config import safe_import_jax
jax, jnp = safe_import_jax()

from rheojax.models.hvm import HVMLocal
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display

plt.rcParams.update({
    'figure.dpi': 100,
    'font.size': 10,
    'axes.labelsize': 11,
    'axes.titlesize': 12,
    'legend.fontsize': 9
})

print(f"JAX version: {jax.__version__}")
print(f"Devices: {jax.devices()}")
import sys, os
sys.path.insert(0, os.path.dirname(os.path.abspath("")))
from utils.plotting_utils import (
    plot_nlsq_fit, display_arviz_diagnostics, plot_posterior_predictive
)

## Theory: Creep Compliance

For a constant applied stress σ₀, the creep compliance is:

$$J(t) = \frac{\gamma(t)}{\sigma_0}$$

The HVM model exhibits:

1. **Elastic jump** at t=0: J(0⁺) = 1/(G_P + G_E + G_D)
2. **Retardation modes**: Exponential relaxation from E and D networks
3. **Long-time plateau**: J(∞) → 1/G_P (permanent network)
4. **Vitrimer plastic creep**: Additional slow flow from TST-mediated bond exchange

The retardation times are:
- τ_E = η_E / G_E (E-network)
- τ_D = G_D / k_d_D (D-network)

where η_E is the TST-activated viscosity of the E-network.

In [ ]:
# Model setup
model = HVMLocal()
model.parameters.set_value("G_P", 5000.0)    # Permanent network modulus [Pa]
model.parameters.set_value("G_E", 3000.0)    # E-network (exchangeable) [Pa]
model.parameters.set_value("G_D", 1000.0)    # D-network (dangling) [Pa]
model.parameters.set_value("T", 350.0)       # Temperature [K]
model.parameters.set_value("k_d_D", 10.0)    # D-network dissociation rate [1/s]
model.parameters.set_value("nu_0", 1e10)     # TST attempt frequency [1/s]
model.parameters.set_value("E_a", 80e3)      # Activation energy [J/mol]
model.parameters.set_value("V_act", 1e-5)    # Activation volume [m³/mol]

print("HVM Model Parameters:")
print(f"  G_P = {model.G_P:.0f} Pa")
print(f"  G_E = {model.G_E:.0f} Pa")
print(f"  G_D = {model.G_D:.0f} Pa")
print(f"  T = {model.T:.0f} K")
print(f"  k_d_D = {model.k_d_D:.1f} 1/s")
print(f"  τ_D = G_D/k_d_D = {model.G_D/model.k_d_D:.1f} s")
print(f"  Long-time compliance: 1/G_P = {1/model.G_P:.2e} 1/Pa")


In [ ]:
# Creep simulation
t = np.logspace(-2, 3, 300)
sigma_0 = 500.0

print(f"Simulating creep at σ₀ = {sigma_0:.0f} Pa...")
result = model.simulate_creep(t, sigma_0=sigma_0, return_full=True)
J_t = np.array(result["strain"]) / sigma_0

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Creep compliance
ax1.loglog(t, J_t, "k-", lw=2, label="J(t)")
ax1.axhline(1/model.G_P, color="blue", ls=":", lw=1.5, 
            label=f"1/G_P = {1/model.G_P:.2e}")
ax1.set_xlabel("t [s]")
ax1.set_ylabel("J(t) [1/Pa]")
ax1.set_title("Creep Compliance")
ax1.legend()
ax1.grid(True, alpha=0.3, which="both")

# Creep strain
ax2.semilogx(t, result["strain"], "k-", lw=2)
ax2.set_xlabel("t [s]")
ax2.set_ylabel("Strain γ")
ax2.set_title("Creep Strain")
ax2.grid(True, alpha=0.3, which="both")

plt.tight_layout()
display(fig)
plt.close(fig)

print(f"\nCreep compliance range: {J_t.min():.2e} to {J_t.max():.2e} 1/Pa")
print(f"Final strain: {result['strain'][-1]:.4f}")

In [ ]:
# Stress-dependent creep
stresses = [200, 500, 1000]
colors = ['blue', 'green', 'red']

fig, ax = plt.subplots(figsize=(8, 5))

print("\nComparing creep at different stress levels...")
for stress, color in zip(stresses, colors):
    r = model.simulate_creep(t, sigma_0=float(stress), return_full=True)
    J = np.array(r["strain"]) / stress
    ax.loglog(t, J, "-", lw=1.5, color=color, label=f"σ₀ = {stress} Pa")
    print(f"  σ₀ = {stress:4.0f} Pa: J(t_max) = {J[-1]:.2e} 1/Pa")

ax.axhline(1/model.G_P, color="black", ls=":", lw=1, alpha=0.5,
           label="1/G_P (theoretical plateau)")
ax.set_xlabel("t [s]")
ax.set_ylabel("J(t) [1/Pa]")
ax.set_title("Creep Compliance at Different Stress Levels")
ax.legend()
ax.grid(True, alpha=0.3, which="both")

plt.tight_layout()
display(fig)
plt.close(fig)

## Key Takeaways

1. **Elastic jump**: Immediate elastic response at $t=0$ from all three networks: $J(0^+) = 1/(G_P + G_E + G_D)$
2. **Retardation**: D-network and E-network relax with characteristic times $\tau_D$ and $\tau_{E,eff}$
3. **Long-time plateau**: Creep compliance approaches $1/G_P$ as only the permanent network remains
4. **Vitrimer plasticity**: TST-activated bond exchange enables slow plastic flow beyond the elastic plateau
5. **Stress dependence**: Higher $\sigma_0$ accelerates TST kinetics (via $V_{act}$ term), enhancing creep

## Further Reading

- [HVM Model Reference](../../docs/source/models/hvm/hvm.rst) — Constitutive equations and evolution equations
- [HVM Knowledge Extraction](../../docs/source/models/hvm/hvm_knowledge.rst) — Application: reprocessing time estimation from $t_{reprocess} \sim 5/k_{BER,0}(T)$

**References:**
1. Vernerey, F.J., Long, R. & Brighenti, R. (2017). *J. Mech. Phys. Solids*, 107, 1-20.
2. Meng, F., Saed, M.O. & Terentjev, E.M. (2019). *Macromolecules*, 52(19), 7423-7429.

## Next Notebooks

- **Notebook 05**: Flow curve — steady-state subnetwork decomposition
- **Notebook 06**: LAOS — nonlinear harmonic generation